In [3]:
!pip install transformers

import pandas as pd
import torch
import transformers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import time


# Load data
df = pd.read_csv("Bug_Reports_Final_data.csv")

# Select the first 50 records
#df = df.head(50)
# Define labels and convert them to numerical values
label_dict = {"low": 0, "medium": 1, "high": 2, "critical": 3}
df['label'] = df['label'].apply(lambda x: label_dict[x])

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['stem'], df['label'], test_size=.2)

# Load tokenizer and encode data
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

# Convert data to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels.tolist())
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_labels.tolist())
)

# Define model and optimizer
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_dict))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Define training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
start_time = time.time()
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

for epoch in range(10):
    # Training
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()

        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    total_preds = []
    total_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(
                input_ids,
                attention_mask=attention_mask,
                labels=None
            )

            logits = outputs.logits

            preds = torch.argmax(logits, axis=1).flatten()

            total_preds.extend(preds.cpu().detach().numpy())
            total_labels.extend(labels.cpu().detach().numpy())

    print(f"Epoch {epoch+1} - Train loss: {avg_train_loss:.2f} - Val classification report:")
    print(classification_report(total_labels, total_preds))
    end_time = time.time()

    total_time = end_time - start_time

    print(f"Total time taken: {total_time:.2f} seconds")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1 - Train loss: 1.10 - Val classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        41
           1       0.31      0.94      0.46        32
           2       0.33      0.08      0.13        48
           3       0.87      0.99      0.92        79

    accuracy                           0.56       200
   macro avg       0.38      0.50      0.38       200
weighted avg       0.47      0.56      0.47       200

Total time taken: 70.05 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2 - Train loss: 0.82 - Val classification report:
              precision    recall  f1-score   support

           0       0.51      0.90      0.65        41
           1       0.75      0.38      0.50        32
           2       0.67      0.33      0.44        48
           3       0.90      0.99      0.94        79

    accuracy                           0.71       200
   macro avg       0.71      0.65      0.63       200
weighted avg       0.74      0.71      0.69       200

Total time taken: 139.79 seconds
Epoch 3 - Train loss: 0.70 - Val classification report:
              precision    recall  f1-score   support

           0       0.92      0.27      0.42        41
           1       0.54      0.84      0.66        32
           2       0.62      0.69      0.65        48
           3       0.92      0.99      0.95        79

    accuracy                           0.74       200
   macro avg       0.75      0.70      0.67       200
weighted avg       0.79      0.74      0